In [1]:
!git clone https://github.com/asanskruti/return_risk.git

fatal: destination path 'return_risk' already exists and is not an empty directory.


In [2]:
import os
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import gzip
import shutil

In [3]:
# loading the dataset
df = pd.read_csv("return_risk/data/amazon_co-ecommerce_sample.csv")

display(df.head())
display(df.info())
display(df.describe())

,index,uniq_id,product_name,manufacturer,price,number_available_in_stock,number_of_reviews,number_of_answered_questions,average_review_rating,amazon_category_and_sub_category,customers_who_bought_this_item_also_bought,description,product_information,product_description,items_customers_buy_after_viewing_this_item,customer_questions_and_answers,customer_reviews,sellers
0,0,eac7efa5dbd3d667f26eb3d3ab504464,Hornby 2014 Catalogue,Hornby,£3.42,5 new,15,1.0,4.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Product Description Hornby 2014 Catalogue Box ...,Technical Details Item Weight640 g Product Dim...,Product Description Hornby 2014 Catalogue Box ...,http://www.amazon.co.uk/Hornby-R8150-Catalogue...,Does this catalogue detail all the previous Ho...,Worth Buying For The Pictures Alone (As Ever) ...,"{""seller""=>[{""Seller_name_1""=>""Amazon.co.uk"", ..."
1,1,b17540ef7e86e461d37f3ae58b7b72ac,FunkyBuys® Large Christmas Holiday Express Fes...,FunkyBuys,£16.99,NaN,2,1.0,4.5 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,Size Name:Large FunkyBuys® Large Christmas Hol...,Technical Details Manufacturer recommended age...,Size Name:Large FunkyBuys® Large Christmas Hol...,http://www.amazon.co.uk/Christmas-Holiday-Expr...,can you turn off sounds // hi no you cant turn...,Four Stars // 4.0 // 18 Dec. 2015 // By\n \...,"{""seller""=>{""Seller_name_1""=>""UHD WHOLESALE"", ..."
2,2,348f344247b0c1a935b1223072ef9d8a,CLASSIC TOY TRAIN SET TRACK CARRIAGES LIGHT EN...,ccf,£9.99,2 new,17,2.0,3.9 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Classic-Train-Lights-B...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,Technical Details Manufacturer recommended age...,BIG CLASSIC TOY TRAIN SET TRACK CARRIAGE LIGHT...,http://www.amazon.co.uk/Train-With-Tracks-Batt...,What is the gauge of the track // Hi Paul.Trut...,**Highly Recommended!** // 5.0 // 26 May 2015 ...,"{""seller""=>[{""Seller_name_1""=>""DEAL-BOX"", ""Sel..."
3,3,e12b92dbb8eaee78b22965d2a9bbbd9f,HORNBY Coach R4410A BR Hawksworth Corridor 3rd,Hornby,£39.99,NaN,1,2.0,5.0 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,NaN,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,Technical Details Item Weight259 g Product Dim...,Hornby 00 Gauge BR Hawksworth 3rd Class W 2107...,NaN,NaN,I love it // 5.0 // 22 July 2013 // By\n \n...,NaN
4,4,e33a9adeed5f36840ccc227db4682a36,Hornby 00 Gauge 0-4-0 Gildenlow Salt Co. Steam...,Hornby,£32.19,NaN,3,2.0,4.7 out of 5 stars,Hobbies > Model Trains & Railway Sets > Rail V...,http://www.amazon.co.uk/Hornby-R6367-RailRoad-...,Product Description Hornby RailRoad 0-4-0 Gild...,Technical Details Item Weight159 g Product Dim...,Product Description Hornby RailRoad 0-4-0 Gild...,http://www.amazon.co.uk/Hornby-R2672-RailRoad-...,NaN,Birthday present // 5.0 // 14 April 2014 // By...,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   index                                        10000 non-null  int64  
 1   uniq_id                                      10000 non-null  object 
 2   product_name                                 10000 non-null  object 
 3   manufacturer                                 9993 non-null   object 
 4   price                                        8565 non-null   object 
 5   number_available_in_stock                    7500 non-null   object 
 6   number_of_reviews                            9982 non-null   object 
 7   number_of_answered_questions                 9235 non-null   float64
 8   average_review_rating                        9982 non-null   object 
 9   amazon_category_and_sub_category             9310 non-null   object 
 10 

None

,index,number_of_answered_questions
count,10000.00000,9235.000000
mean,4999.50000,1.834976
std,2886.89568,2.517268
min,0.00000,1.000000
25%,2499.75000,1.000000
50%,4999.50000,1.000000
75%,7499.25000,2.000000
max,9999.00000,39.000000


In [4]:
print(df.columns.tolist())

['index', 'uniq_id', 'product_name', 'manufacturer', 'price', 'number_available_in_stock', 'number_of_reviews', 'number_of_answered_questions', 'average_review_rating', 'amazon_category_and_sub_category', 'customers_who_bought_this_item_also_bought', 'description', 'product_information', 'product_description', 'items_customers_buy_after_viewing_this_item', 'customer_questions_and_answers', 'customer_reviews', 'sellers']


In [5]:
# drop fully empty rows, columns and duplicates
df.dropna(how = "all" ,inplace = True)
df.dropna(axis = 1, how = "all", inplace = True)
df.drop_duplicates(inplace=True)

In [6]:
# convert price to numeric (remove currency symbols)
df['price'] = (
    pd.to_numeric(
        df['price'].astype(str).str.replace(r'[^\d.]', '', regex=True),
        errors='coerce'
    )
)

# convert rating and review-related columns to numeric
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce')
df['number_of_answered_questions'] = pd.to_numeric(df['number_of_answered_questions'], errors='coerce')
df['average_review_rating'] = (
    df['average_review_rating']
    .astype(str)
    .str.extract(r'([\d.]+)')[0]
    .astype(float)
)

# clean stock values
df['number_available_in_stock'] = (
    df['number_available_in_stock']
    .astype(str)
    .str.extract(r'(\d+)')[0]
    .astype(float)
)

# input median values for missing values
for col in ['price', 'number_of_reviews', 'number_of_answered_questions',
            'average_review_rating', 'number_available_in_stock']:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

In [7]:
# Price bins — categorize price into bands
df['price_bin'] = pd.cut(
    df['price'],
    bins=[0, 20, 50, 100, 200, 500, np.inf],
    labels=['£0–20', '£20–50', '£50–100', '£100–200', '£200–500', '£500+']
)

In [8]:
# add return status based on average review rating and random products from specific price bins
rating_threshold = 3.5
price_bins_for_random_return = ['£0–20', '£200–500', '£500+']
random_sample_fraction = 0.3

# Condition 1: average review rating <= rating_threshold
condition_rating = (df['average_review_rating'] <= rating_threshold)

# Condition 2: random products from specified price bins
condition_price_bin = False
for price_bin in price_bins_for_random_return:
    # Get indices of products in the current price bin
    price_bin_indices = df[df['price_bin'] == price_bin].index
    # Randomly sample a fraction of these indices
    random_indices = np.random.choice(
        price_bin_indices,
        size=int(len(price_bin_indices) * random_sample_fraction),
        replace=False
    )
    # Create a boolean mask for the random indices
    random_mask = df.index.isin(random_indices)
    condition_price_bin = condition_price_bin | random_mask

# Combine conditions
df['return_status'] = (condition_rating | condition_price_bin).astype(int)

In [10]:
# Split data into training and test sets based on index
train_data = df.iloc[:4000].copy() # Use .copy() to avoid SettingWithCopyWarning
test_data = df.iloc[4000:].copy() # Use .copy() to avoid SettingWithCopyWarning

# Define output directory
output_dir = r'return_risk/data/processed_data'
os.makedirs(output_dir, exist_ok=True)

# Save training and test data
train_path = os.path.join(output_dir, 'train_data.csv')
train_data.to_csv(train_path, index=False)
test_path = os.path.join(output_dir, 'test_data.csv')
test_data.to_csv(test_path, index=False)

✅ Training data saved at: return_risk/data/processed_data/train_data.csv
✅ Test data saved at: return_risk/data/processed_data/test_data.csv


In [18]:
nltk.download('stopwords')
default_stopwords = set(stopwords.words('english'))
custom_words = {
    'him', 'her', 'his', 'hers', 'this', 'that', 'these', 'those',
    'it', 'its', 'my', 'your', 'yours', 'our', 'ours', 'their', 'theirs',
    'he', 'she', 'they', 'them', 'we', 'us'
}

all_stopwords = default_stopwords.union(custom_words)

def clean_review_custom(text):
    if pd.isnull(text):
        return ""
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize and remove stopwords + custom words
    words = text.split()
    words = [word for word in words if word not in all_stopwords]
    # Join back to string
    return " ".join(words)

# Apply text cleaning to customer reviews in train_data
train_data['customer_reviews_clean'] = train_data['customer_reviews'].astype(str).apply(clean_review_custom)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# Apply text cleaning to relevant text columns in train_data
text_cols_to_clean = ['product_name', 'manufacturer', 'amazon_category_and_sub_category', 'description', 'product_information', 'product_description']

for col in text_cols_to_clean:
    if col in train_data.columns:
        train_data[col + '_clean'] = train_data[col].astype(str).apply(clean_review_custom)

display(train_data[[col + '_clean' for col in text_cols_to_clean if col + '_clean' in train_data.columns]].head())

,product_name_clean,manufacturer_clean,amazon_category_and_sub_category_clean,description_clean,product_information_clean,product_description_clean
0,hornby catalogue,hornby,hobbies model trains railway sets rail vehicle...,product description hornby catalogue box conta...,technical details item weight g product dimens...,product description hornby catalogue box conta...
1,funkybuys® large christmas holiday express fes...,funkybuys,hobbies model trains railway sets rail vehicle...,size namelarge funkybuys® large christmas holi...,technical details manufacturer recommended age...,size namelarge funkybuys® large christmas holi...
2,classic toy train set track carriages light en...,ccf,hobbies model trains railway sets rail vehicle...,big classic toy train set track carriage light...,technical details manufacturer recommended age...,big classic toy train set track carriage light...
3,hornby coach ra br hawksworth corridor rd,hornby,hobbies model trains railway sets rail vehicle...,hornby gauge br hawksworth rd class w w ra,technical details item weight g product dimens...,hornby gauge br hawksworth rd class w w ra
4,hornby gauge gildenlow salt co steam locomotiv...,hornby,hobbies model trains railway sets rail vehicle...,product description hornby railroad gildenlow ...,technical details item weight g product dimens...,product description hornby railroad gildenlow ...


In [20]:
# sentiment analysis in customer reviews and other feature engineering on train_data
def get_sentiment(text):
    if pd.isna(text):
        return 0
    return TextBlob(text).sentiment.polarity

# Apply sentiment analysis to the cleaned customer reviews
train_data['review_sentiment'] = train_data['customer_reviews_clean'].apply(get_sentiment)

def label_sentiment(score):
    if score < -0.2:
        return 'Negative'
    elif score > 0.2:
        return 'Positive'
    else:
        return 'Neutral'

train_data['review_sentiment_label'] = train_data['review_sentiment'].apply(label_sentiment)

#rating bin
def rating_bin(rating):
    if pd.isna(rating):
        return 'Unknown'
    elif rating < 2.5:
        return 'Low'
    elif rating < 4.0:
        return 'Medium'
    else:
        return 'High'

train_data['rating_bin'] = train_data['average_review_rating'].apply(rating_bin)

#category risk flag
def category_risk(cat):
    if pd.isna(cat):
        return 'Unknown'
    cat = cat.lower()
    high_risk_keywords = ['dress', 'jeans', 'shirt', 'blouse', 'shoes', 'pants', 'trousers', 'skirt']
    for word in high_risk_keywords:
        if word in cat:
            return 'High'
    return 'Low'

train_data['category_risk'] = train_data['amazon_category_and_sub_category'].apply(category_risk)

# description length
train_data['description_length'] = train_data['description'].astype(str).apply(len)

def description_bin(length):
    if length < 50:
        return 'Short'
    elif length < 200:
        return 'Medium'
    else:
        return 'Long'

train_data['description_bin'] = train_data['description_length'].apply(description_bin)

#known brand flag
trusted_brands = ['Nike', 'Adidas', 'Zara', 'H&M', 'Levi', 'Uniqlo', 'Puma', 'North Face']

def is_known_brand(manu):
    if pd.isna(manu):
        return 0
    for brand in trusted_brands:
        if brand.lower() in manu.lower():
            return 1
    return 0

train_data['known_brand'] = train_data['manufacturer'].apply(is_known_brand)

#engagement bin
def engagement_bin(num):
    if num == 0:
        return 'None'
    elif num < 3:
        return 'Low'
    elif num < 10:
        return 'Medium'
    else:
        return 'High'

train_data['engagement'] = train_data['number_of_answered_questions'].apply(engagement_bin)

#cross-sell length
train_data['cross_sell_length'] = train_data['customers_who_bought_this_item_also_bought'].astype(str).apply(len)

display(train_data[['review_sentiment', 'review_sentiment_label', 'rating_bin', 'category_risk', 'description_length', 'description_bin', 'known_brand', 'engagement', 'cross_sell_length']].head())

,review_sentiment,review_sentiment_label,rating_bin,category_risk,description_length,description_bin,known_brand,engagement,cross_sell_length
0,0.372154,Positive,Low,Low,72,Medium,0,Low,367
1,0.800000,Positive,Low,Low,134,Medium,0,Low,489
2,0.276351,Positive,Low,Low,357,Long,0,Low,445
3,0.500000,Positive,Low,Low,57,Medium,0,Low,3
4,0.309583,Positive,Low,Low,398,Long,0,Low,422


In [21]:
# Ordinal encode rating bin
rating_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Unknown': -1}
train_data['rating_bin_num'] = train_data['rating_bin'].map(rating_map)

# Ordinal encode category risk
category_map = {'Low': 0, 'High': 1, 'Unknown': -1}
train_data['category_risk_num'] = train_data['category_risk'].map(category_map)

# Ordinal encode description bin
desc_map = {'Short': 0, 'Medium': 1, 'Long': 2}
train_data['description_bin_num'] = train_data['description_bin'].map(desc_map)

# One-hot encode sentiment label
train_data = pd.get_dummies(train_data, columns=['review_sentiment_label'], prefix='sentiment')

# Ordinal encode engagement
engagement_map = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3}
train_data['engagement_num'] = train_data['engagement'].map(engagement_map)

display(train_data[['rating_bin_num', 'category_risk_num', 'description_bin_num', 'sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive', 'engagement_num']].head())

,rating_bin_num,category_risk_num,description_bin_num,sentiment_Negative,sentiment_Neutral,sentiment_Neutral,sentiment_Positive,engagement_num
0,0,0,1,False,True,False,True,1
1,0,0,1,False,True,False,True,1
2,0,0,2,False,True,False,True,1
3,0,0,1,False,True,False,True,1
4,0,0,2,False,True,False,True,1


In [22]:
from sklearn.preprocessing import StandardScaler

# Identify numerical columns to scale
numerical_cols = [
    'price',
    'number_available_in_stock',
    'number_of_reviews',
    'average_review_rating',
    'description_length',
    'cross_sell_length',
    'review_sentiment',
    'number_of_answered_questions'
]

# Drop columns that are not in the DataFrame
numerical_cols = [col for col in numerical_cols if col in train_data.columns]

# Initialize the scaler
scaler = StandardScaler()

# Apply scaling to the numerical columns of train_data
train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])

display(train_data[numerical_cols].head())

,price,number_available_in_stock,number_of_reviews,average_review_rating,description_length,cross_sell_length,review_sentiment,number_of_answered_questions
0,-0.430040,-0.235694,0.203194,0.517295,-0.676984,0.286252,-0.151121,-0.292944
1,-0.032633,-0.235694,-0.218774,-0.538813,-0.526501,1.075815,1.710241,-0.292944
2,-0.237633,-0.635288,0.268112,-2.122975,0.014750,0.791055,-0.567917,0.006284
3,0.640939,-0.235694,-0.251233,0.781322,-0.713391,-2.069493,0.405079,0.006284
4,0.412510,-0.235694,-0.186315,-0.010759,0.114263,0.642203,-0.423337,0.006284


In [29]:
# Define features (X) and target (y) for the training data
features = [
    'price',
    'number_available_in_stock',
    'number_of_reviews',
    'number_of_answered_questions',
    'average_review_rating',
    'review_sentiment',
    'description_length',
    'cross_sell_length',
    'rating_bin_num',
    'category_risk_num',
    'description_bin_num',
    'known_brand',
    'engagement_num',
    'sentiment_Negative',
    'sentiment_Neutral',
    'sentiment_Positive'
]

# Drop features that are not in the DataFrame
features = [col for col in features if col in train_data.columns]


X_train = train_data[features]
y_train = train_data['return_status']

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (4000, 16)
Shape of y_train: (4000,)


In [30]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model using the training data
model.fit(X_train, y_train)

print("✅ Logistic Regression model trained successfully!")

✅ Logistic Regression model trained successfully!


In [31]:
# Apply text cleaning to customer reviews in test_data
test_data['customer_reviews_clean'] = test_data['customer_reviews'].astype(str).apply(clean_review_custom)

# Apply text cleaning to other relevant text columns in test_data
text_cols_to_clean = ['product_name', 'manufacturer', 'amazon_category_and_sub_category', 'description', 'product_information', 'product_description']

for col in text_cols_to_clean:
    if col in test_data.columns:
        test_data[col + '_clean'] = test_data[col].astype(str).apply(clean_review_custom)

display(test_data[['customer_reviews_clean'] + [col + '_clean' for col in text_cols_to_clean if col + '_clean' in test_data.columns]].head())

,customer_reviews_clean,product_name_clean,manufacturer_clean,amazon_category_and_sub_category_clean,description_clean,product_information_clean,product_description_clean
4000,great july alannah july lovely model realistic...,schleich tinker mare,schleich,figures playsets farm animals,product description tinker beautiful piebald c...,technical details item weight g product dimens...,product description tinker beautiful piebald c...
4001,five stars jan amazon customer jan sturdy son ...,bruder jeep wrangler unlimited rubicon police ...,bruder,figures playsets farm animals,product description bruder known vast selectio...,technical details item weight kg product dimen...,product description bruder known vast selectio...
4002,happy july natasha hood july beautiful little ...,tales legends princess sissi figure cm hand pa...,papo,figures playsets knights castles,princess sissi,technical details item weight g product dimens...,princess sissi
4003,merlin ultimate fan feb woolley feb huge mega ...,adventures merlin gwen guinevere pendragon lim...,scificollector,fancy dress accessories,guinevere pendragon latest range merlin inch a...,technical details item weight g product dimens...,guinevere pendragon latest range merlin inch a...
4004,fisher price imaginext castle friends knight e...,fisher price imaginext castle friends knight e...,mattel,figures playsets knights castles,medieval castle without group knights defend c...,technical details item weight g product dimens...,medieval castle without group knights defend c...


In [32]:
# sentiment analysis and other feature engineering on test_data
def get_sentiment(text):
    if pd.isna(text):
        return 0
    return TextBlob(text).sentiment.polarity

# Apply sentiment analysis to the cleaned customer reviews
if 'customer_reviews_clean' in test_data.columns:
    test_data['review_sentiment'] = test_data['customer_reviews_clean'].apply(get_sentiment)
else:
     print("Warning: 'customer_reviews_clean' not found in test_data. Skipping sentiment analysis.")
     test_data['review_sentiment'] = 0

# Use the same label_sentiment function defined for training data
test_data['review_sentiment_label'] = test_data['review_sentiment'].apply(label_sentiment)

# Use the same rating_bin function and mapping defined for training data
test_data['rating_bin'] = test_data['average_review_rating'].apply(rating_bin)

# Use the same category_risk function and mapping defined for training data
test_data['category_risk'] = test_data['amazon_category_and_sub_category'].apply(category_risk)

# description length
test_data['description_length'] = test_data['description'].astype(str).apply(len)

# Use the same description_bin function defined for training data
test_data['description_bin'] = test_data['description_length'].apply(description_bin)

# Use the same is_known_brand function defined for training data
test_data['known_brand'] = test_data['manufacturer'].apply(is_known_brand)

# Use the same engagement_bin function defined for training data
test_data['engagement'] = test_data['number_of_answered_questions'].apply(engagement_bin)

# cross-sell length
test_data['cross_sell_length'] = test_data['customers_who_bought_this_item_also_bought'].astype(str).apply(len)


display(test_data[['review_sentiment', 'review_sentiment_label', 'rating_bin', 'category_risk', 'description_length', 'description_bin', 'known_brand', 'engagement', 'cross_sell_length']].head())

,review_sentiment,review_sentiment_label,rating_bin,category_risk,description_length,description_bin,known_brand,engagement,cross_sell_length
4000,0.402222,Positive,High,Low,155,Medium,0,Low,415
4001,0.000000,Neutral,High,Low,427,Long,0,Low,392
4002,0.534722,Positive,High,Low,14,Short,0,Low,424
4003,0.377647,Positive,High,High,233,Long,0,Low,227
4004,0.202778,Positive,High,Low,690,Long,0,Low,469


In [33]:
# Ordinal encode rating bin using the same mapping
rating_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Unknown': -1}
test_data['rating_bin_num'] = test_data['rating_bin'].map(rating_map)

# Ordinal encode category risk using the same mapping
category_map = {'Low': 0, 'High': 1, 'Unknown': -1}
test_data['category_risk_num'] = test_data['category_risk'].map(category_map)

# Ordinal encode description bin using the same mapping
desc_map = {'Short': 0, 'Medium': 1, 'Long': 2}
test_data['description_bin_num'] = test_data['description_bin'].map(desc_map)

# One-hot encode sentiment label using the same columns as train data
test_data = pd.get_dummies(test_data, columns=['review_sentiment_label'], prefix='sentiment')

# Ensure all sentiment columns from training data are present in test data, fill missing with False
for col in ['sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive']:
    if col not in test_data.columns:
        test_data[col] = False


# Ordinal encode engagement using the same mapping
engagement_map = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3}
test_data['engagement_num'] = test_data['engagement'].map(engagement_map)

display(test_data[['rating_bin_num', 'category_risk_num', 'description_bin_num', 'sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive', 'engagement_num']].head())

,rating_bin_num,category_risk_num,description_bin_num,sentiment_Negative,sentiment_Neutral,sentiment_Positive,engagement_num
4000,2,0,1,False,False,True,1
4001,2,0,2,False,True,False,1
4002,2,0,0,False,False,True,1
4003,2,1,2,False,False,True,1
4004,2,0,2,False,False,True,1


In [34]:
# Identify numerical columns to scale (same as training data)
numerical_cols = [
    'price',
    'number_available_in_stock',
    'number_of_reviews',
    'average_review_rating',
    'description_length',
    'cross_sell_length',
    'review_sentiment',
    'number_of_answered_questions'
]

# Drop columns that are not in the DataFrame
numerical_cols = [col for col in numerical_cols if col in test_data.columns]

# Apply the *same* scaler fitted on the training data to the numerical columns of test_data
test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])

display(test_data[numerical_cols].head())

,price,number_available_in_stock,number_of_reviews,average_review_rating,description_length,cross_sell_length,review_sentiment,number_of_answered_questions
4000,-0.354776,1.895474,-0.121397,-1.330894,-0.475531,0.596900,-0.020308,-0.292944
4001,0.488946,2.961058,-0.251233,0.781322,0.184650,0.448048,-1.770192,-0.292944
4002,-0.302647,0.963088,-0.251233,0.781322,-0.817758,0.655146,0.556139,-0.292944
4003,-0.179062,-0.368892,-0.088938,0.253268,-0.286215,-0.619804,-0.127223,-0.292944
4004,-0.220940,0.297098,-0.088938,0.781322,0.822987,0.946379,-0.887999,-0.292944


In [35]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Define features (X_test) and target (y_test) for the test data
features = [
    'price',
    'number_available_in_stock',
    'number_of_reviews',
    'number_of_answered_questions',
    'average_review_rating',
    'review_sentiment',
    'description_length',
    'cross_sell_length',
    'rating_bin_num',
    'category_risk_num',
    'description_bin_num',
    'known_brand',
    'engagement_num',
    'sentiment_Negative',
    'sentiment_Neutral',
    'sentiment_Positive'
]

# Ensure test_data has the same columns as X_train
X_test = test_data[features]
y_test = test_data['return_status']


# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86      4557
           1       0.42      0.00      0.01      1443

    accuracy                           0.76      6000
   macro avg       0.59      0.50      0.43      6000
weighted avg       0.68      0.76      0.66      6000


Confusion Matrix:
[[4550    7]
 [1438    5]]

ROC AUC Score:
0.6165983170591465


# Task
Wrap the code in the notebook into functions for data loading and cleaning, data splitting, data preprocessing, model training, and model testing/evaluation. Update the notebook to use these functions to execute the workflow. Ensure the functions are designed to facilitate training and evaluating multiple models and comparing their results.

## Create a data loading and initial cleaning function

### Subtask:
Wrap the code for loading the dataset and performing initial cleaning steps (handling missing values, basic type conversions) into a function that takes the file path as input and returns the cleaned DataFrame.


**Reasoning**:
Define a function to load and perform initial cleaning on the data as per the instructions.



In [36]:
def load_and_clean_data(file_path):
    """
    Loads the dataset and performs initial cleaning steps.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    df = pd.read_csv(file_path)

    # Drop fully empty rows, columns and duplicates
    df.dropna(how="all", inplace=True)
    df.dropna(axis=1, how="all", inplace=True)
    df.drop_duplicates(inplace=True)

    # convert price to numeric (remove currency symbols)
    df['price'] = (
        pd.to_numeric(
            df['price'].astype(str).str.replace(r'[^\d.]', '', regex=True),
            errors='coerce'
        )
    )

    # convert rating and review-related columns to numeric
    df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce')
    df['number_of_answered_questions'] = pd.to_numeric(df['number_of_answered_questions'], errors='coerce')
    df['average_review_rating'] = (
        df['average_review_rating']
        .astype(str)
        .str.extract(r'([\d.]+)')[0]
        .astype(float)
    )

    # clean stock values
    df['number_available_in_stock'] = (
        df['number_available_in_stock']
        .astype(str)
        .str.extract(r'(\d+)')[0]
        .astype(float)
    )

    # input median values for missing values
    for col in ['price', 'number_of_reviews', 'number_of_answered_questions',
                'average_review_rating', 'number_available_in_stock']:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

    return df

# Example usage (will be replaced by the main workflow later)
# cleaned_df = load_and_clean_data("return_risk/data/amazon_co-ecommerce_sample.csv")
# display(cleaned_df.head())
# display(cleaned_df.info())

## Create a data splitting function

### Subtask:
Wrap the code for splitting the DataFrame into training and test sets based on index into a function that takes the DataFrame and split index as input and returns the training and test DataFrames.


**Reasoning**:
Define the function `split_data` to split the DataFrame into training and test sets based on the provided index.



In [37]:
def split_data(df, split_index):
    """
    Splits the DataFrame into training and test sets based on index.

    Args:
        df (pd.DataFrame): The input DataFrame.
        split_index (int): The index at which to split the DataFrame.

    Returns:
        tuple: A tuple containing the training DataFrame and the test DataFrame.
    """
    train_data = df.iloc[:split_index].copy()
    test_data = df.iloc[split_index:].copy()
    return train_data, test_data

# Example usage (will be replaced by the main workflow later)
# split_index = 4000
# train_data, test_data = split_data(df, split_index)
# print("Shape of train_data:", train_data.shape)
# print("Shape of test_data:", test_data.shape)

## Create a preprocessing function

### Subtask:
Wrap the code for applying all preprocessing steps (text cleaning, feature engineering, encoding, scaling) into a function. This function should take a DataFrame (either train or test) as input and return the preprocessed DataFrame. It will need to handle fitting transformers on the training data and applying the fitted transformers to both train and test data.


**Reasoning**:
Define the preprocess_data function including all the specified data cleaning, feature engineering, encoding, and scaling steps.



In [ ]:
def preprocess_data(df_input, scaler=None):
    """
    Applies all preprocessing steps (text cleaning, feature engineering,
    encoding, scaling) to a DataFrame.

    Args:
        df_input (pd.DataFrame): The input DataFrame (train or test).
        scaler (StandardScaler, optional): A fitted scaler object to use
                                           for transforming numerical columns.
                                           Defaults to None (for fitting on train data).

    Returns:
        pd.DataFrame: The preprocessed DataFrame.
        StandardScaler (optional): The fitted scaler object, returned only
                                   when scaler is None (for training data).
    """
    df = df_input.copy() # Work on a copy to avoid modifying the original DataFrame

    # Define helper functions (as specified in the instructions)
    # These are defined inside the main function to keep them self-contained,
    # although in a larger project they might be defined outside.
    def clean_review_custom(text):
        if pd.isnull(text):
            return ""
        # Lowercase
        text = text.lower()
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Tokenize and remove stopwords + custom words
        words = text.split()
        words = [word for word in words if word not in all_stopwords] # all_stopwords is defined globally
        # Join back to string
        return " ".join(words)

    def get_sentiment(text):
        if pd.isna(text):
            return 0
        return TextBlob(text).sentiment.polarity

    def label_sentiment(score):
        if score < -0.2:
            return 'Negative'
        elif score > 0.2:
            return 'Positive'
        else:
            return 'Neutral'

    def rating_bin(rating):
        if pd.isna(rating):
            return 'Unknown'
        elif rating < 2.5:
            return 'Low'
        elif rating < 4.0:
            return 'Medium'
        else:
            return 'High'

    def category_risk(cat):
        if pd.isna(cat):
            return 'Unknown'
        cat = str(cat).lower() # Ensure cat is a string
        high_risk_keywords = ['dress', 'jeans', 'shirt', 'blouse', 'shoes', 'pants', 'trousers', 'skirt']
        for word in high_risk_keywords:
            if word in cat:
                return 'High'
        return 'Low'

    def description_bin(length):
        if length < 50:
            return 'Short'
        elif length < 200:
            return 'Medium'
        else:
            return 'Long'

    def is_known_brand(manu):
        if pd.isna(manu):
            return 0
        trusted_brands = ['Nike', 'Adidas', 'Zara', 'H&M', 'Levi', 'Uniqlo', 'Puma', 'North Face']
        for brand in trusted_brands:
            if brand.lower() in str(manu).lower(): # Ensure manu is a string
                return 1
        return 0

    def engagement_bin(num):
        if pd.isna(num) or num == 0:
            return 'None'
        elif num < 3:
            return 'Low'
        elif num < 10:
            return 'Medium'
        else:
            return 'High'


    # Apply text cleaning
    text_cols_to_clean = ['customer_reviews', 'product_name', 'manufacturer', 'amazon_category_and_sub_category', 'description', 'product_information', 'product_description']
    for col in text_cols_to_clean:
        if col in df.columns:
            df[col + '_clean'] = df[col].astype(str).apply(clean_review_custom)

    # Sentiment analysis
    if 'customer_reviews_clean' in df.columns:
        df['review_sentiment'] = df['customer_reviews_clean'].apply(get_sentiment)
    else:
         df['review_sentiment'] = 0 # Default to 0 if column is missing

    df['review_sentiment_label'] = df['review_sentiment'].apply(label_sentiment)

    # Feature engineering
    df['rating_bin'] = df['average_review_rating'].apply(rating_bin)
    df['category_risk'] = df['amazon_category_and_sub_category'].apply(category_risk)
    df['description_length'] = df['description'].astype(str).apply(len)
    df['description_bin'] = df['description_length'].apply(description_bin)
    df['known_brand'] = df['manufacturer'].apply(is_known_brand)
    df['engagement'] = df['number_of_answered_questions'].apply(engagement_bin)
    df['cross_sell_length'] = df['customers_who_bought_this_item_also_bought'].astype(str).apply(len)


    # Encoding
    rating_map = {'Low': 0, 'Medium': 1, 'High': 2, 'Unknown': -1}
    df['rating_bin_num'] = df['rating_bin'].map(rating_map)

    category_map = {'Low': 0, 'High': 1, 'Unknown': -1}
    df['category_risk_num'] = df['category_risk'].map(category_map)

    desc_map = {'Short': 0, 'Medium': 1, 'Long': 2}
    df['description_bin_num'] = df['description_bin'].map(desc_map)

    # One-hot encode sentiment label
    df = pd.get_dummies(df, columns=['review_sentiment_label'], prefix='sentiment')
    # Ensure all sentiment columns are present
    for col in ['sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive']:
        if col not in df.columns:
            df[col] = False # Use False for boolean representation

    engagement_map = {'None': 0, 'Low': 1, 'Medium': 2, 'High': 3}
    df['engagement_num'] = df['engagement'].map(engagement_map)

    # Scaling numerical columns
    numerical_cols = [
        'price',
        'number_available_in_stock',
        'number_of_reviews',
        'number_of_answered_questions',
        'average_review_rating',
        'review_sentiment',
        'description_length',
        'cross_sell_length'
    ]

    # Drop columns that are not in the DataFrame
    numerical_cols = [col for col in numerical_cols if col in df.columns]

    if scaler is None:
        # Fit and transform on training data
        current_scaler = StandardScaler()
        df[numerical_cols] = current_scaler.fit_transform(df[numerical_cols])
        return df, current_scaler
    else:
        # Transform on test data using the provided scaler
        df[numerical_cols] = scaler.transform(df[numerical_cols])
        return df

# Example usage (will be replaced by the main workflow later)
# train_data_processed, fitted_scaler = preprocess_data(train_data)
# test_data_processed = preprocess_data(test_data, scaler=fitted_scaler)

# display(train_data_processed[['rating_bin_num', 'category_risk_num', 'description_bin_num', 'sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive', 'engagement_num']].head())
# display(train_data_processed[numerical_cols].head())
# display(test_data_processed[['rating_bin_num', 'category_risk_num', 'description_bin_num', 'sentiment_Negative', 'sentiment_Neutral', 'sentiment_Positive', 'engagement_num']].head())
# display(test_data_processed[numerical_cols].head())
